<h1>STOCK DATA MINING AND INTERPRETATION REPORT</h1>

This notebook is a compilation of the work done for crawling stock related information, data exploration and interpretation. 

Each of the section begins with a milestone video and the python code used to generate some of the data in the video follows.

In [0]:
#@title Install all required packages
#install all required packages
!pip install IPython
!pip install requests_html
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install feedparser
!pip install requests
!pip install lxml

In [33]:
#@title Milestone 1 Video
from IPython.display import HTML
import io, base64
video=io.open('data/ms1.mp4','r+b').read()
encoded=base64.b64encode(video)
HTML(data="""<video width="640" height="480" controls><source src="data:video/mp4;base64,{0}" type="video/mp4"></video>""".format(encoded.decode('ascii')))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
#@title MS1: Initialization
#@markdown This will import all necessary packages
from requests_html import HTMLSession
import datetime
import pandas as pd
from sqlalchemy import create_engine
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


In [0]:
#@title MS1: Stocks data crawling
#@markdown This code will crawl for stocks data from www.thestar.com.my/business/marketwatch/stock-list as shown in the MS1 video
def getdata(url):
    #function to get stockname and price from thestar site
    #session=HTMLSession()
    #r=session.get(url)
    #r.html.render(wait=2)
    wd = webdriver.Chrome('chromedriver',options=options)
    wd.get(url)
    elements=wd.find_elements_by_xpath("//tr[@class='linedlist']//td")
    lines=[]
    for elem in elements:
      lines.append(elem.text)
    
    print('number of lines=',len(lines))
    #initlialize list
    stock, openp, highp,lowp, lastp = ([] for i in range(0,5))
    
    for i in range(0,int(len(lines)/8)):
        if len(re.sub('[^0-9]','',lines[i*8+4]))==0:
            break
        else:
            stock.append(lines[i*8])
            openp.append(lines[i*8+1])
            highp.append(lines[i*8+2])
            lowp.append(lines[i*8+3])
            lastp.append(lines[i*8+4])
    
    return stock, openp, highp, lowp, lastp
    
inlist=['A','B','C','D','E','F','G','H','I','J', 'K','L','M','N','O','P','Q','R','S','T', 'U','V','W','X','Y','Z','0-9'] 

name, op, hi, lo, last= ([] for i in range(0,5))
tdate = datetime.date.today().isoformat()

for item in inlist:
    df=pd.DataFrame()
    url="https://www.thestar.com.my/business/marketwatch/stock-list/?alphabet="+item
    print(url)
    a, b, c, d, e = getdata(url)
    if len(a)==len(b) and len(b)==len(c) and len(c)==len(d) and len(d)==len(a):
        for i,j,k,l,m in zip(a,b,c,d,e):
	        name.append(i)
	        op.append(j)
	        hi.append(k)
	        lo.append(l)
	        last.append(m)
    else:
        print("ERROR: not all lists are equal length")

    #create dataframe of stock price
    df=pd.DataFrame({'Date':tdate, 'Name':name, 'Open':op, 'High':hi, 'Low':lo, 'Last':last})
    #df.to_csv(filename, encoding='utf-8', index=False)
    print(df)


In [0]:
#@title MS1: Script to crawl for stock and business news headlines
import feedparser
from dateutil.parser import parse
import requests
import lxml.html as lh

def getrss(url):
    newsfeed = feedparser.parse(url)
    result = newsfeed.entries
    return result

def getnews(stocknum, news_url):
    url=news_url+stocknum
    page=requests.get(url)
    root=lh.fromstring(page.content)
    charset='iso-8859-1'
    #table=lh.tostring(root.xpath('//*[@id="ctl17_tbCorpHeadline"]')[0], encoding='iso-8859-1')
    
    try:
        tblink = root.xpath('//*[@id="ctl17_tbCorpHeadline"]')[0]
        date = [l.encode(charset).decode('utf8') for l in tblink.xpath('//tr[@class="line"]//td/text()')]
        title = [l.xpath('text()')[0].encode(charset).decode('utf8') for l in tblink.xpath('//tr[@class="line"]//a[contains(@href,"newsID")]')]
        href = [l.encode(charset).decode('utf8') for l in tblink.xpath('//tr[@class="line"]//a[contains(@href,"newsID")]/@href')]
        print('INFO: Getting stock ',stocknum)
        stack=[[stocknum,x,y,z] for x,y,z in zip(date, title, href)]
        df=pd.DataFrame(stack, columns=['Code' ,'UpdateDate','Title','Link'])
    #some stock codes don't exist or the page exists but there are no news items
    #in this case, return empty dataframe and print error
    except:
        df=pd.DataFrame(columns=['Code','UpdateDate','Title','Link'])
        print("ERROR: could not get news items for %s" %stocknum)
    return df

def main():
    output_to_csv = False
    rss_source = {'thestarbusiness':'http://www.thestar.com.my/rss/business/business-news/',
              'malaysiakini':'https://www.malaysiakini.com/en/columns.rss',
              'theedge':'http://www.theedgemarkets.com/mymalaysia.rss',
              'sunbusiness':'https://www.thesundaily.my/rss/business'}

    news_url = 'https://www.malaysiastock.biz/Corporate-Infomation.aspx?securityCode='
    stocklist_df=pd.read_csv('data/stockinfo2.csv')
    today = datetime.date.today()
    timestamp = pd.Timestamp.utcnow()
    
    #initialize main variables
    all_rss = []
    rssDF = pd.DataFrame(columns=['Code','UpdateDate','Title','Link'])
    newsDF = pd.DataFrame(columns=['Code','UpdateDate','Title','Link'])
    
    #get rss news and only keep those that are from current date
    for source in rss_source.keys():
        for item in getrss(rss_source[source]):
            print('INFO: Getting rss from ',rss_source[source])
            if parse(item['published']).date()==today:
                all_rss.append(item)
            
    #save all_rss into rssDF dataframe
    for i in range(0,len(all_rss)):
        rssDF.loc[i] = [None ,timestamp ,all_rss[i]['title'],all_rss[i]['links'][0]['href']]
    
    #get news headlines for each stock code
    for code in list(stocklist_df['Code']):
        tmpdf = getnews(str(code),news_url)
        for i in range(0, len(tmpdf['UpdateDate'])):
            if list(tmpdf['UpdateDate'])[i]==today.strftime('%d %b'):
                tmpdf['UpdateDate'][i] = timestamp
                newsDF = newsDF.append(tmpdf.iloc[[i]])
    
    finalDF = pd.concat([newsDF, rssDF], sort=False)
    
main()

In [32]:
#@title Milestone 2 Video
from IPython.display import HTML
import io, base64
video=io.open('data/ms2.mp4','r+b').read()
encoded=base64.b64encode(video)
HTML(data="""<video width="640" height="480" controls><source src="data:video/mp4;base64,{0}" type="video/mp4"></video>""".format(encoded.decode('ascii')))

Output hidden; open in https://colab.research.google.com to view.

In [49]:
#@title Milestone 3 Video
from IPython.display import HTML
import io, base64
video=io.open('data/ms3.mp4','r+b').read()
encoded=base64.b64encode(video)
HTML(data="""<video width="640" height="480" controls><source src="data:video/mp4;base64,{0}" type="video/mp4"></video>""".format(encoded.decode('ascii')))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
#@title MS3: Pairwise correlation for stocks
#@markdown This code loads in the combined stockprice and index daily data and performs pairwise correlation
df_stockindices=pd.read_csv('data/stockprice_indices.csv')
#get correlation matrix
df_corr=df_stockindices.iloc[:,1:].corr()
print(df_corr)


In [51]:
#@title Milestone 4 Video
from IPython.display import HTML
import io, base64
video=io.open('data/ms4.mp4','r+b').read()
encoded=base64.b64encode(video)
HTML(data="""<video width="640" height="480" controls><source src="data:video/mp4;base64,{0}" type="video/mp4"></video>""".format(encoded.decode('ascii')))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
#@title MS4: Sentiment score calculation
#@markdown This code shows the sentiment score computation based on NLTK for a small sample of news headlines.
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
def get_polarity(t):
    sid = SentimentIntensityAnalyzer()
    neg=sid.polarity_scores(t)['neg']
    pos=sid.polarity_scores(t)['pos']
    #return neg,pos
    print(t,'\t-'+str(neg),'\t+'+str(pos),'\t')
    
news=['Alliance Bank 3Q net profit up 21.5% on-year at RM149m','Quick take: AirAsia falls 7.5% as analysts downgrade stock','AirAsia active, falls 5.92% on 4Q loss, downgrade','Trading ideas: Genting, Genting Malaysia, UMW, Padini, AirAsia','HLIB Research downgrades AirAsia, lowers target price to RM3.20','CIMB Research retains Reduce for AirAsia, TP RM1.82','AirAsia falls to first quarterly loss since 2015 as fuel costs rise']

for n in news:
  get_polarity(n)

In [57]:
#@title Milestone 5 Video
from IPython.display import HTML
import io, base64
video=io.open('data/ms5.mp4','r+b').read()
encoded=base64.b64encode(video)
HTML(data="""<video width="640" height="480" controls><source src="data:video/mp4;base64,{0}" type="video/mp4"></video>""".format(encoded.decode('ascii')))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
#@title MS5: Code for labeling buy opportunity points
import pandas as pd

days_window=3
profit=1.1

incsv='data/labelingdata.csv'
dfstocks=pd.read_csv(incsv)

#list of all stocks
stocklist=list(set(list(dfstocks['Name'])))

dt=[]
#convert dates to datetime
for d in list(dfstocks['Date']):
    dt.append(datetime.datetime.strptime(d,"%d/%m/%Y"))
dfstocks['Datetime']=dt

#subsetting
dfstocks.iloc[list(dfstocks['Name']==dfstocks['Name'][0]),:][['Date','Datetime','Last Price']]

def flatten(inlist):
    olist=[]
    for list1 in inlist:
        for item in list1:
            olist.append(item)
    return olist

#master list
list_date=[]
list_datetime=[]
list_name=[]
list_lastprice=[]
list_tag=[]

for name in dfstocks['Name']:
    dftmp=dfstocks.iloc[list(dfstocks['Name']==name),:][['Date','Datetime','Name','Last Price']]
    dftmp=dftmp.sort_values('Datetime')
    d=list(dftmp['Datetime'])
    lp=list(dftmp['Last Price'])
    d2=list(dftmp['Date'])
    n=list(dftmp['Name'])
    
    tag=[]
    
    for i in list(set(d)):
        tag.append("Don't Buy")
        
    for i in range(0,len(d)-days_window):
        base=lp[i]
        if base>0:
            for j in range(1, days_window):
                ratio=lp[i+j]/base
                if ratio>1.1:
                    tag[i]="buy opportunity"
                
    list_date.append(d2)
    list_datetime.append(d)
    list_name.append(n)
    list_lastprice.append(lp)
    list_tag.append(tag)
    
list_date=flatten(list_date)
list_datetime=flatten(list_datetime)
list_name=flatten(list_name)
list_lastprice=flatten(list_lastprice)
list_tag=flatten(list_tag)

dfresult=pd.DataFrame({'Date':list_date, 'Datetime':list_datetime, 'Name':list_name,'Last Price':list_lastprice,'Tag':list_tag})
print(dfresult)